In [1]:
import pandas as pd
import numpy as np
import re
import logging
from pathlib import Path

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.info("🚀 전처리 시작!")

# 1. 엑셀 or 기존 csv 로드
file_path = "company.xlsx"  # 또는 "company.csv"
if file_path.lower().endswith(".pdf"):
    raise ValueError("PDF 아니라고 했잖아!!!")

df = pd.read_excel(file_path) if file_path.endswith(".xlsx") else pd.read_csv(file_path, encoding="utf-8")

# 2. 컬럼명 변경: 잔업시간 → overtime_hours
if "잔업시간" in df.columns:
    df = df.rename(columns={"잔업시간": "overtime_hours"})

# 3. IT기업용 팀구분 매핑 (한글 노이즈 보정 포함)
team_mapping = {
    "인사": "HR",
    "총무": "General Affairs",
    "기획": "Planning",
    "마켓팅": "Marketing",
    "마케팅": "Marketing",
    "재무": "Finance",
    "개발": "Development",
    "영업": "Sales",
    "법무": "Legal",
    "개발팀": "Development",
    "개발실": "Development",
}

df["팀구분"] = df["팀구분"].astype(str).str.strip().replace(team_mapping)

# 4. 필요 컬럼만 남기기
keep_columns = [
    "사번", "이름", "성별", "나이", "근속기간",
    "입사년도", "직군", "팀구분", "퇴직구분",
    "overtime_hours", "성과점수", "미사용.휴가일수", "연봉"
]
df = df[[col for col in keep_columns if col in df.columns]]

# 5. 텍스트 Noise 제거 (but analysis엔 중요 데이터는 보존🔥)
df = df.replace(r"={3,}", "", regex=True)
df = df.replace(r"-{3,}", "", regex=True)
df = df.replace(r"\|{3,}", "", regex=True)
df = df.replace(r"_{3,}", "", regex=True)

# 6. 수치 컬럼 NaN 안정 처리
df["나이"] = pd.to_numeric(df["나이"], errors="coerce").fillna(0).astype(int)
df["입사년도"] = pd.to_numeric(df["입사년도"], errors="coerce").fillna(df["입사년도"].mode()[0] if not df["입사년도"].isna().all() else 2020).astype(int)
df["근속기간"] = pd.to_numeric(df["근속기간"], errors="coerce").fillna(0).astype(int)
df["overtime_hours"] = pd.to_numeric(df["overtime_hours"], errors="coerce").fillna(0).astype(int)
df["성과점수"] = pd.to_numeric(df["성과점수"], errors="coerce").fillna(df["성과점수"].median() if not df["성과점수"].isna().all() else 50).astype(int)
df["연봉"] = pd.to_numeric(df["연봉"], errors="coerce").fillna(df["연봉"].median() if not df["연봉"].isna().all() else 4000)

# 7. 최종 CSV 저장
output_csv = "processed_company_data.csv"
df.to_csv(output_csv, index=False, encoding="utf-8")
logger.info(f"✅ CSV 저장 완료! → {output_csv}")

df.head()


ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [5]:
import pandas as pd
import numpy as np
import re
import logging
from pathlib import Path

logger = logging.getLogger()
logging.basicConfig(level=logging.INFO)
logger.info("🚀 IT 직원 데이터 전처리 시작!")

file_path = "company.csv"

# 1. CSV 로드 (인코딩 자동 fallback)
try:
    df = pd.read_csv(file_path, encoding="utf-8")
    logger.info("✅ UTF-8 로딩 성공!")
except UnicodeDecodeError:
    df = pd.read_csv(file_path, encoding="cp949", encoding_errors="ignore")
    logger.info("✅ CP949 로딩 성공! (UTF-8 아님)")

# 2. "초과근무" 표기 rename
if "잔업시간" in df.columns:
    df = df.rename(columns={"잔업시간": "overtime_hours"})

# 3. 민감한 특수문자/노이즈 제거 먼저 실행
noise_patterns = [r"={3,}", r"-{3,}", r"\|{3,}", r"_{3,}", r"~{3,}", r"={2,}PAGE_BREAK={2,}"]
for p in noise_patterns:
    df = df.replace(p, "", regex=True)

df = df.replace(r"[ \t]{2,}", " ", regex=True)

# 4. 팀구분 매핑을 너 회사 기준으로
team_mapping = {
    "인사": "HR",
    "총무": "General Affairs",
    "총무팀": "General Affairs",
    "기획": "Planning",
    "마케팅": "Marketing",
    "마켓팅": "Marketing",
    "재무": "Finance",
    "재무팀": "Finance",
    "개발": "Development",
    "개발팀": "Development",
    "영업": "Sales",
    "영업팀": "Sales",
    "법무": "Legal",
    "법무팀": "Legal"
}
df["팀구분"] = df["팀구분"].astype(str).str.strip().replace(team_mapping)

# 5. 안정처리 할 numeric 컬럼들(절대 key만 존재하는 이름으로)
numeric_columns = ["나이", "근속기간", "입사년도", "overtime_hours", "성과점수", "연봉", "연봉"]  # 중복 name 방지

# 하지만 실제 df 기준으로 존재하는 것만 골라서 안정처리
for col in ["나이", "근속기간", "입사년도", "overtime_hours", "성과점수", "연봉"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce").fillna(
            0 if col != "성과점수" else 50
        ).astype(int)

# 6. 남겨야 할 컬럼만 정확히 유지
keep_cols_final = [
    "사번", "이름", "성별", "나이", "근속기간",
    "입사년도", "직군", "팀구분", "퇴직구분",
    "overtime_hours", "성과점수", "미사용.휴가일수", "연봉"
]

df = df[[col for col in keep_cols_final if col in df.columns]]

# 7. CSV 저장
output_csv = "processed_company_data.csv"
df.to_csv(output_csv, index=False, encoding="utf-8")
logger.info(f"🎉✅ 전처리 완료! 최종 CSV 저장됨 → {output_csv}")

df.head()


INFO:root:🚀 IT 직원 데이터 전처리 시작!
INFO:root:✅ CP949 로딩 성공! (UTF-8 아님)
INFO:root:🎉✅ 전처리 완료! 최종 CSV 저장됨 → processed_company_data.csv


,사번,이름,성별,나이,근속기간,입사년도,직군,팀구분,퇴직구분,overtime_hours,성과점수,미사용.휴가일수,연봉
0,4006,한경택,남,39,19,2015,제조생산,마케팅팀,No,213,42,14,66002987
1,3337,임경화,여,33,17,2012,마케팅영업,기술팀,No,324,49,9,58479133
2,2543,이규경,남,46,27,2013,제조생산,생산팀,No,192,40,22,46179481
3,3999,윤규만,남,39,19,2015,마케팅영업,사업팀,No,20,51,14,61002987
4,1365,주강필,남,32,13,2011,경영지원,인력팀,Yes,201,32,22,83001494


In [9]:
import pandas as pd
import re
import logging
import random
from pathlib import Path

logger = logging.getLogger()
logging.basicConfig(level=logging.INFO)
logger.info("🚀 직원 데이터 전처리 + 자동 부서 분류 시작!")

df = pd.read_csv("company.csv", encoding="cp949", encoding_errors="ignore")

# ---------------------------------------------------
# 1. 컬럼명 보정 (잔업 관련)
# ---------------------------------------------------
if "overtime_hours" in df.columns:
    df = df.rename(columns={"overtime_hours": "초과근무"})
if "잔업시간" in df.columns:
    df = df.rename(columns={"잔업시간": "초과근무"})

# ---------------------------------------------------
# 2. 자동 부서 분류 함수 (내가 알아서 때려맞춤)
# ---------------------------------------------------
def auto_team_classify(val):
    if not val:
        return None

    val = val.strip()
    val_clean = re.sub(r"\s+", "", val)

    # ============= AI 기반 키워드 판단 =============
    if any(k in val_clean for k in ["dev", "개발", "engine", "백엔드", "프론트"]):
        return "개발"
    if any(k in val_clean for k in ["hr", "인사", "recruit", "채용"]):
        return "인사"
    if any(k in val_clean for k in ["admin", "총무", "ga", "시설", "운영지원"]):
        return "총무"
    if any(k in val_clean for k in ["plan", "기획", "strategy", "pm", "컨설팅"]):
        return "기획"
    if any(k in val_clean for k in ["market", "마케팅", "pr", "광고"]):
        return "마켓팅"
    if any(k in val_clean for k in ["finance", "재무", "회계", "budget"]):
        return "재무"
    if any(k in val_clean for k in ["sales", "영업", "biz", "사업"]):
        return "영업"
    if any(k in val_clean for k in ["law", "법무", "compliance", "규정"]):
        return "법무"

    # ============= 아무도 모르겠으면 =============
    return None

df["team_auto"] = df["팀구분"].apply(auto_team_classify)

# ---------------------------------------------------
# 3. 균형 맞춰서 None 값들 랜덤 분배
# ---------------------------------------------------
valid_teams = ["인사", "총무", "기획", "마켓팅", "재무", "개발", "영업", "법무"]

# 현재 분류된 팀 개수 계산
current_counts = df["team_auto"].value_counts().to_dict()

def balanced_random_assign(counts, options):
    # 제일 적게 배정된 부서 우선 선택
    min_team = min(options, key=lambda t: counts.get(t, 0))
    return min_team

# None 값 채우기
for idx in df[df["team_auto"].isnull()].index:
    assigned = balanced_random_assign(current_counts, valid_teams)
    df.at[idx, "team_auto"] = assigned
    current_counts[assigned] = current_counts.get(assigned, 0) + 1

# 최종 team_auto → team구분 컬럼으로 옮김
df["팀구분"] = df["team_auto"]
df = df.drop(columns=["team_auto", "team_auto"])

# 불필요 컬럼 정리
df = df[[c for c in ["사번","이름","성별","나이","근속기간","입사년도","직군","팀구분","퇴직구분","초과근무","성과점수","미사용.휴가일수","연봉"] if c in df.columns]]


df.to_csv("company_final_departments.csv", index=False, encoding="utf-8")
logger.info("🎉✅ AI 자동 부서 분류 끝! → company_final_departments.csv")
df.head()


INFO:root:🚀 직원 데이터 전처리 + 자동 부서 분류 시작!
INFO:root:🎉✅ AI 자동 부서 분류 끝! → company_final_departments.csv


,사번,이름,성별,나이,근속기간,입사년도,직군,팀구분,퇴직구분,초과근무,성과점수,미사용.휴가일수,연봉
0,4006,한경택,남,39,19.5,2015,제조생산,마켓팅,No,213,42.4,14,66002987
1,3337,임경화,여,33,17.9,2012,마케팅영업,인사,No,324,49.9,9,58479133
2,2543,이규경,남,46,27.0,2013,제조생산,총무,No,192,40.0,22,46179481
3,3999,윤규만,남,39,19.9,2015,마케팅영업,영업,No,20,51.8,14,61002987
4,1365,주강필,남,32,13.9,2011,경영지원,기획,Yes,201,32.6,22,83001494


In [20]:
import pandas as pd
import re
import logging
import random

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

logger.info("🚀 직원 데이터 전처리 + 자동 부서/직급 분류 시작!")

# 1. CSV 로드
df = pd.read_csv("company.csv", encoding="cp949", encoding_errors="ignore")

# 2. 초과근무 컬럼 한국어 보정
if "overtime_hours" in df.columns:
    df = df.rename(columns={"overtime_hours": "초과근무"})
if "잔업시간" in df.columns:
    df = df.rename(columns={"잔업시간": "초과근무"})
if "초과근무" not in df.columns:
    df["초과근무"] = 0

# 3. 자동 부서 분류 함수
def auto_team_classify(val):
    if not val or pd.isna(val):
        return None
    val = val.strip()
    val_clean = re.sub(r"\s+", "", val.lower())
    if any(k in val_clean for k in ["dev","개발","engine","백엔드","프론트"]):
        return "개발"
    if any(k in val_clean for k in ["hr","인사","recruit","채용"]):
        return "인사"
    if any(k in val_clean for k in ["admin","총무","시설","운영지원","ga"]):
        return "총무"
    if any(k in val_clean for k in ["plan","기획","strategy","pm","컨설팅"]):
        return "기획"
    if any(k in val_clean for k in ["market","마케팅","pr","광고"]):
        return "마켓팅"
    if any(k in val_clean for k in ["finance","재무","회계","budget"]):
        return "재무"
    if any(k in val_clean for k in ["sales","영업","biz","사업"]):
        return "영업"
    if any(k in val_clean for k in ["law","법무","compliance","규정"]):
        return "법무"
    return None

# ❗팀구분 자동 수정 적용❗
df["팀구분"] = df["팀구분"].apply(auto_team_classify)

# 결측 → valid 부서 중 하나로 채우기
valid_teams = ["인사","총무","기획","마켓팅","재무","개발","영업","법무"]

for idx in df[df["팀구분"].isnull()].index:
    df.at[idx, "팀구분"] = random.choice(valid_teams)

logger.info("✅ 팀구분 자동 균형 배정 완료!")

# --------------------------------------------------
# 👑 4. 직급 자동 인원 균형 배치 (너가 준 기준 정확 적용)
# --------------------------------------------------
grade_order = ["인턴","사원","주임","대리","과장","차장","부장","이사","상무","전무","대표이사"]
weights = [600,900,800,700,500,200,150,80,40,20,1]  # 대표이사는 1명만

# 직급 컬럼 먼저 생성
df["직급"] = None
logger.info("🔥 '직급' 컬럼 생성 완료!")

# 대표이사 1명만
ceo_idxs = df[df["팀구분"] == "대표이사"].index  # 혹시 부서에 CEO 박힌 경우도 대비
real_ceo = df[df["직급"] == "대표이사"].index

if len(real_ceo) > 1:
    df.at[real_ceo[0], "직급"] = "대표이사"
    for i in real_ceo[1:]:
        df.at[i, "직급"] = None
    logger.info("👑 대표이사 1명 제한 적용!")

# 직급 균형배정
for idx in df[df["직급"].isnull()].index:
    df.at[idx, "직급"] = random.choices(grade_order, weights=weights, k=1)[0]

logger.info("✅ 직급 균형 자동배정 완료!")

# 5. 수치 컬럼 int로 안정 변환
for col in ["나이","근속기간","입사년도","성과점수","연봉","초과근무"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0).astype(int)

# 6. 최종 필요한 컬럼 정리
keep_cols = ["사번","이름","성별","나이","근속기간","입사년도","직급","팀구분","퇴직구분","초과근무","성과점수","미사용.휴가일수","연봉"]
for c in keep_cols:
    if c not in df.columns:
        df[c] = ""

df = df[keep_cols]

# 7. 최종 CSV 저장
df.to_csv("company_final_departments.csv", index=False, encoding="utf-8")
logger.info("🎉✅ 전처리 완료 → company_final_departments.csv")

df.head()


INFO:root:🚀 직원 데이터 전처리 + 자동 부서/직급 분류 시작!
INFO:root:✅ 팀구분 자동 균형 배정 완료!
INFO:root:🔥 '직급' 컬럼 생성 완료!
INFO:root:✅ 직급 균형 자동배정 완료!
INFO:root:🎉✅ 전처리 완료 → company_final_departments.csv


,사번,이름,성별,나이,근속기간,입사년도,직급,팀구분,퇴직구분,초과근무,성과점수,미사용.휴가일수,연봉
0,4006,한경택,남,39,19,2015,주임,마켓팅,No,213,42,14,66002987
1,3337,임경화,여,33,17,2012,인턴,개발,No,324,49,9,58479133
2,2543,이규경,남,46,27,2013,주임,법무,No,192,40,22,46179481
3,3999,윤규만,남,39,19,2015,과장,영업,No,20,51,14,61002987
4,1365,주강필,남,32,13,2011,대리,인사,Yes,201,32,22,83001494


In [35]:
import pandas as pd
import re
import logging
import os
import random

logger = logging.getLogger()
logging.basicConfig(level=logging.INFO)
logger.info("🚀 CSV 로딩 + 자동 부서/직급(grade) + 연봉 구조 보정 시작!")

# ---------------------------
# 1. CSV 로드
# ---------------------------
file_path = "company.csv"
df = pd.read_csv(file_path, encoding="cp949", encoding_errors="ignore")
logger.info(f"✅ 로딩 완료! (rows={len(df)})")

# ---------------------------
# 2. 컬럼명 통일
# ---------------------------
if "직군" in df.columns:
    df = df.rename(columns={"직군": "직급"})
logger.info("✅ 직군 → 직급 rename 완료!")

if "overtime_hours" in df.columns:
    df = df.rename(columns={"overtime_hours": "초과근무"})
if "잔업시간" in df.columns:
    df = df.rename(columns={"잔업시간": "초과근무"})
logger.info("✅ 잔업 관련 → 초과근무 컬럼 통일 완료!")

# ---------------------------
# 3. 팀구분 자동 분류 (8개 한글 부서 유지)
# ---------------------------
valid_teams = ["인사", "총무", "기획", "마켓팅", "재무", "개발", "영업", "법무"]

def classify_team(val):
    if not val or pd.isna(val):
        return None
    v = re.sub(r"\s+", "", str(val).lower())
    if any(k in v for k in ["dev", "backend", "frontend", "engine", "개발"]):
        return "개발"
    if any(k in v for k in ["sales", "biz", "사업", "영업"]):
        return "영업"
    if any(k in v for k in ["hr", "recruit", "인사", "채용"]):
        return "인사"
    if any(k in v for k in ["admin", "총무", "시설", "운영지원"]):
        return "총무"
    if any(k in v for k in ["plan", "strategy", "pm", "기획", "컨설팅"]):
        return "기획"
    if any(k in v for k in ["market", "marketing", "pr", "광고", "마켓팅"]):
        return "마켓팅"
    if any(k in v for k in ["finance", "회계", "budget", "재무"]):
        return "재무"
    if any(k in v for k in ["법무", "legal", "law", "규정"]):
        return "법무"
    return None

if "팀구분" in df.columns:
    mapped = df["팀구분"].apply(classify_team)
    df["팀구분"] = mapped.fillna(df["팀구분"])
else:
    df["팀구분"] = None

df["팀구분"] = df["팀구분"].apply(lambda x: x if x in valid_teams else None)

counts = df["팀구분"].value_counts().to_dict()
for idx in df[df["팀구분"].isnull()].index:
    t = min(valid_teams, key=lambda k: counts.get(k, 0))
    df.at[idx, "팀구분"] = t
    counts[t] = counts.get(t, 0) + 1

logger.info("✅ 팀구분 자동 균형 분포 완료!")

# ---------------------------
# 4. 연봉 기준 직급 피라미드 배정
# ---------------------------
grade_order = ["인턴","사원","주임","대리","과장","차장","부장","이사","상무","전무","대표이사"]

df["연봉"] = pd.to_numeric(df["연봉"], errors="coerce").fillna(df["연봉"].median() or 4000).astype(int)

# 대표이사 1명 고정 (최고 연봉자)
ceo_idx = df["연봉"].idxmax()
df.loc[:, "직급"] = None
df.at[ceo_idx, "직급"] = "대표이사"
logger.info("👑 대표이사 1명 지정 완료!")

# 나머지 10개 직급은 연봉 낮은 순서대로 할당
others_idx = df[df["직급"].isnull()].index
salary_rank = df.loc[others_idx, "연봉"].sort_values().index.tolist()

sections = {}
start = 0
n = len(salary_rank)

weights = [0.09,0.43,0.23,0.14,0.05,0.025,0.008,0.004,0.002,0.001]  # CEO 제외 10개 직급
for i, w in enumerate(weights):
    size = int((n) * w)
    sections[i] = size

diff = n - sum(sections.values())
for i in range(diff):
    sections[i % len(sections)] += 1

start = 0
for i, cnt in sections.items():
    end = start + cnt - 1
    grade = grade_order[i]  # 대표이사 제외 10개
    for rank_idx in salary_rank[start:end+1]:
        df.at[rank_idx, "직급"] = grade
    start = end + 1

logger.info("✅ 연봉 낮은 순 → 직급 자동 배정 완료!")

# ---------------------------
# 5. 연봉–직급 침범 제거 (각 직급 범위 내에서만 유지)
# ---------------------------
exec_grades = ["전무","상무","이사","부장","차장","과장","대리","주임","사원","인턴"]

# 직급별 최솟값/최댓값 연봉 찾기
for grade in exec_grades:
    subset = df[df["직급"] == grade]["연봉"]
    if len(subset) > 0:
        df.loc[df["직급"] == grade, "연봉"] = df.loc[df["직급"] == grade, "연봉"].clip(subset.min(), subset.max())

logger.info("✅ 모든 직급의 연봉 침범 제거 완료!")

# ---------------------------
# 6. 수치 컬럼 안정 처리
# ---------------------------
df["나이"] = pd.to_numeric(df["나이"], errors="coerce").fillna(df["나이"].median() or 0).astype(int)
df["근속기간"] = pd.to_numeric(df["근속기간"], errors="coerce").fillna(0).astype(int)
df["초과근무"] = pd.to_numeric(df["초과근무"], errors="coerce").fillna(0).astype(int)
df["성과점수"] = pd.to_numeric(df["성과점수"], errors="coerce").fillna(df["성과점수"].median() or 50).astype(int)

# ---------------------------
# 7. 최종 컬럼 유지
# ---------------------------
keep_final = ["사번","이름","성별","나이","근속기간","입사년도","직급","팀구분","퇴직구분","초과근무","성과점수","미사용.휴가일수","연봉"]
df = df[[c for c in keep_final if c in df.columns]]

# ---------------------------
# 8. 저장 (파일 잠금 대비)
# ---------------------------
final_file = "company_final_cleaned.csv"
if os.path.exists(final_file):
    try:
        os.remove(final_file)
        logger.info("🔥 기존 결과파일 삭제 완료!")
    except PermissionError:
        logger.warning("⚠ 기존 결과파일이 열려 있음 → 닫고 다시 실행해야 저장됨!")

df.to_csv(final_file, index=False, encoding="utf-8")
logger.info(f"🎉✅ 최종 저장 완료! → {final_file}")

print(df.head())
print("\n📊 직급 분포 ↓")
print(df["직급"].value_counts())


INFO:root:🚀 CSV 로딩 + 자동 부서/직급(grade) + 연봉 구조 보정 시작!
INFO:root:✅ 로딩 완료! (rows=2017)
INFO:root:✅ 직군 → 직급 rename 완료!
INFO:root:✅ 잔업 관련 → 초과근무 컬럼 통일 완료!
INFO:root:✅ 팀구분 자동 균형 분포 완료!
INFO:root:👑 대표이사 1명 지정 완료!
INFO:root:✅ 연봉 낮은 순 → 직급 자동 배정 완료!
INFO:root:✅ 모든 직급의 연봉 침범 제거 완료!
INFO:root:🎉✅ 최종 저장 완료! → company_final_cleaned.csv


     사번   이름 성별  나이  근속기간  입사년도  직급  팀구분 퇴직구분  초과근무  성과점수  미사용.휴가일수        연봉
0  4006  한경택  남  39    19  2015  사원   인사   No   213    42        14  66002987
1  3337  임경화  여  33    17  2012  사원   총무   No   324    49         9  58479133
2  2543  이규경  남  46    27  2013  사원   기획   No   192    40        22  46179481
3  3999  윤규만  남  39    19  2015  사원   영업   No    20    51        14  61002987
4  1365  주강필  남  32    13  2011  주임  마켓팅  Yes   201    32        22  83001494

📊 직급 분포 ↓
직급
사원      871
주임      468
대리      287
인턴      186
과장      104
차장       54
부장       20
이사       12
상무        8
전무        6
대표이사      1
Name: count, dtype: int64


In [36]:
import os
import logging
import re
import random
import pandas as pd
from pathlib import Path
from unidecode import unidecode

logger = logging.getLogger()
logging.basicConfig(level=logging.INFO)
logger.info("🚀 CSV 로딩 + 신규 컬럼(이메일/전화번호/생년월일/주민번호) 생성 시작!")

# ---------------------------------------------------
# 1. CSV 로드
# ---------------------------------------------------
df = pd.read_csv("company_final_cleaned.csv", encoding="utf-8", encoding_errors="ignore")
logger.info(f"✅ 데이터 로딩됨 (rows={len(df)})")

# ---------------------------------------------------
# 2. 이메일 생성 (영문 prefix + 도메인 통일)
# ---------------------------------------------------
EMAIL_DOMAIN = "gmail.com"

def gen_email_prefix(name):
    if not name or pd.isna(name):
        prefix = "employee"
    else:
        name = unidecode(name)
        prefix = re.sub(r'[^a-zA-Z0-9]', '', name)[:8].lower()

    number = random.randint(1000, 99999)
    return f"{prefix}{number}@{EMAIL_DOMAIN}"

df["이메일"] = df["이름"].apply(gen_email_prefix)
logger.info("✅ 이메일 컬럼 생성 완료!")

# ---------------------------------------------------
# 3. 생년월일 생성 (나이와 일치하도록 구성!)
# ---------------------------------------------------
CURRENT_YEAR = 2025

def gen_birth_date(age):
    birth_year = CURRENT_YEAR - age
    month = random.randint(1, 12)
    day = random.randint(1, 28)  # 날짜 안정 범위
    return f"{birth_year}-{month:02d}-{day:02d}"

df["생년월일"] = df["나이"].apply(gen_birth_date)
logger.info("✅ 생년월일(=나이 연동) 생성 완료!")

# ---------------------------------------------------
# 4. 주민등록번호 생성 (앞 6자리 = 생년월일 기반!)
# ---------------------------------------------------
def gen_rrn(birth_date):
    birth = birth_date.replace("-","")  # YYYYMMDD
    ymd = birth[2:8]  # YYMMDD
    gender_digit = random.choice(["1","2","3","4"])
    back_digits = "".join([str(random.randint(0,9)) for _ in range(6)])
    return f"{ymd}-{gender_digit}{back_digits}"

df["주민등록번호"] = df["생년월일"].apply(gen_rrn)
logger.info("✅ 주민등록번호 생성 완료!")

# ---------------------------------------------------
# 5. 전화번호 생성 (도메인·회사 지역 상관없이 랜덤, 형식 통일!)
# ---------------------------------------------------
def gen_phone():
    start = random.choice(["010"])
    mid = random.randint(1000, 9999)
    end = random.randint(1000, 9999)
    return f"{start}-{mid}-{end}"

df["전화번호"] = [gen_phone() for _ in range(len(df))]
logger.info("✅ 전화번호 생성 완료!")

# ---------------------------------------------------
# 6. CSV 저장 (파일 잠금 대비 삭제 후 저장)
# ---------------------------------------------------
output_csv = "company_final_with_sensitive.csv"

if Path(output_csv).exists():
    try:
        os.remove(output_csv)
        logger.info("🔥 기존 결과파일 삭제 성공!")
    except PermissionError:
        logger.warning("⚠ 기존 결과파일이 열려있어 삭제 실패 → 닫고 다시 실행!")

df.to_csv(output_csv, index=False, encoding="utf-8")
logger.info("🎉✅ 최종 저장 완료! → company_final_with_sensitive.csv")

df.head()


INFO:root:🚀 CSV 로딩 + 신규 컬럼(이메일/전화번호/생년월일/주민번호) 생성 시작!
INFO:root:✅ 데이터 로딩됨 (rows=2017)
INFO:root:✅ 이메일 컬럼 생성 완료!
INFO:root:✅ 생년월일(=나이 연동) 생성 완료!
INFO:root:✅ 주민등록번호 생성 완료!
INFO:root:✅ 전화번호 생성 완료!
INFO:root:🔥 기존 결과파일 삭제 성공!
INFO:root:🎉✅ 최종 저장 완료! → company_final_with_sensitive.csv


,사번,이름,성별,나이,근속기간,입사년도,직급,팀구분,퇴직구분,초과근무,성과점수,미사용.휴가일수,연봉,이메일,생년월일,주민등록번호,전화번호
0,4006,한경택,남,39,19,2015,사원,인사,No,213,42,14,66002987,hangyeon72112@gmail.com,1986-09-21,860921-3933039,010-3258-9666
1,3337,임경화,여,33,17,2012,사원,총무,No,324,49,9,58479133,imgyeong88296@gmail.com,1992-06-08,920608-4942233,010-9435-4789
2,2543,이규경,남,46,27,2013,사원,기획,No,192,40,22,46179481,igyugyeo63506@gmail.com,1979-11-18,791118-4861402,010-8710-3117
3,3999,윤규만,남,39,19,2015,사원,영업,No,20,51,14,61002987,yungyuma71328@gmail.com,1986-08-19,860819-2149679,010-5261-3262
4,1365,주강필,남,32,13,2011,주임,마켓팅,Yes,201,32,22,83001494,jugangpi2985@gmail.com,1993-06-26,930626-2513462,010-2982-6038


In [37]:
df = df.drop(columns=["생년월일"])
df = df.drop(columns=["퇴직구분"])
df.to_csv("company_final_list.csv", index=False, encoding="utf-8")

In [38]:
import os
import re
import logging
import random
import pandas as pd
from pathlib import Path
from unidecode import unidecode

logger = logging.getLogger()
logging.basicConfig(level=logging.INFO)
logger.info("🚀 CSV 로딩 + 신규 컬럼(이메일/전화번호/주민번호) 마스킹 시작!")

INPUT_FILE = "company_final_list.csv"
EMAIL_DOMAIN = "gmail.com"

try:
    df = pd.read_csv(INPUT_FILE, encoding="utf-8", encoding_errors="ignore")
    logger.info("✅ UTF-8 로딩 성공!")
except UnicodeDecodeError:
    df = pd.read_csv(INPUT_FILE, encoding="cp949", encoding_errors="ignore")
    logger.info("✅ CP949 fallback 로딩 성공!")

df = df.reset_index(drop=True)
logger.info(f"✅ 데이터 로딩됨 (rows={len(df)})")

# ---------------------------------------------------
# 🪪 사번 마스킹 → USER_01 같은 형식으로 변경 (자동 확장)
# ---------------------------------------------------
def mask_emp_no(idx):
    # 1부터 시작하는 연번으로 → 최소 2자리, 3자리, 4자리... 자동 확장
    num_str = str(idx)
    pad_len = max(2, len(num_str))  # 최소 2자리 이상 유지
    return f"USER_{num_str.zfill(pad_len)}"

if "사번" in df.columns:
    df["사번"] = df.index.map(lambda i: mask_emp_no(i + 1))
    logger.info("✅ 사번 마스킹 완료! (USER_01 형식 적용)")
else:
    logger.error("❌ '사번' 컬럼 없음! 컬럼명을 다시 확인해!")

# ---------------------------------------------------
# 👤 이름 마스킹 → Employee_0000 형식
# ---------------------------------------------------
def mask_name(idx):
    return f"Employee_{idx:04d}"

df["이름"] = df.index.map(mask_name)
logger.info("✅ 이름 마스킹 완료! (Employee_0000 형식 적용)")

# ---------------------------------------------------
# 📧 이메일 마스킹 (existing 컬럼 덮어쓰기)
# local part 3글자 유지, domain 통일
# ---------------------------------------------------
def mask_email(email):
    if pd.isna(email) or str(email).strip() == "":
        return f"***@{EMAIL_DOMAIN}"
    local = str(email).strip().lower().split("@")[0]
    local_clean = re.sub(r'[^a-z0-9]', '', local)
    if not local_clean:
        local_clean = "emp"
    return local_clean[:3] + "****@" + EMAIL_DOMAIN

if "이메일" in df.columns:
    df["이메일"] = df["이메일"].apply(mask_email)
    logger.info("✅ 이메일 마스킹 완료!")
else:
    df["이메일"] = df.index.map(lambda i: f"emp{i:04d}****@{EMAIL_DOMAIN}")
    logger.warning("⚠ 이메일 컬럼 없어서 자동 생성 후 마스킹!")

# ---------------------------------------------------
# 📱 전화번호 마스킹 (앞3자리 유지, 뒤 ****-****)
# ---------------------------------------------------
def mask_phone(phone):
    if pd.isna(phone) or str(phone).strip()=="":
        return "010-****-****"
    digits = re.sub(r'[^0-9]', '', str(phone))
    if len(digits) < 8:
        return "010-****-****"
    return f"{digits[:3]}-****-****"

if "전화번호" in df.columns:
    df["전화번호"] = df["전화번호"].apply(mask_phone)
    logger.info("✅ 전화번호 마스킹 완료!")
else:
    df["전화번호"] = "010-****-****"
    logger.warning("⚠ 전화번호 컬럼 없어서 default 값 적용!")

# ---------------------------------------------------
# 🧾 주민등록번호 마스킹 (앞 6자리 유지 + -*******)
# ---------------------------------------------------
def mask_rrn_val(rrn):
    if pd.isna(rrn) or str(rrn).strip()=="":
        return "******-*******"
    digits = re.sub(r'[^0-9]', '', str(rrn))
    if len(digits) < 6:
        return "******-*******"
    return digits[:6] + "-*******"

if "주민등록번호" in df.columns:
    df["주민등록번호"] = df["주민등록번호"].apply(mask_rrn)
    logger.info("✅ 주민등록번호 마스킹 완료!")
else:
    df["주민등록번호"] = "******-*******"
    logger.warning("⚠ 주민등록번호 컬럼 없어서 default 값 적용!")

# ---------------------------------------------------
# 💾 최종 CSV 저장
# ---------------------------------------------------
OUTPUT_FILE = "company_final_masked.csv"
try:
    df.to_csv(OUTPUT_FILE, index=False, encoding="utf-8")
    logger.info(f"🎉✅ 최종 마스킹 완료 CSV 저장 → {OUTPUT_FILE}")
except PermissionError:
    fallback = "company_final_masked_copy.csv"
    df.to_csv(fallback, index=False, encoding="utf-8")
    logger.warning(f"⚠ 파일 잠김 → 새 파일 저장! → {fallback}")

df.head()


INFO:root:🚀 CSV 로딩 + 신규 컬럼(이메일/전화번호/주민번호) 마스킹 시작!
INFO:root:✅ UTF-8 로딩 성공!
INFO:root:✅ 데이터 로딩됨 (rows=2017)
INFO:root:✅ 사번 마스킹 완료! (USER_01 형식 적용)
INFO:root:✅ 이름 마스킹 완료! (Employee_0000 형식 적용)
INFO:root:✅ 이메일 마스킹 완료!


INFO:root:✅ 전화번호 마스킹 완료!
INFO:root:✅ 주민등록번호 마스킹 완료!
INFO:root:🎉✅ 최종 마스킹 완료 CSV 저장 → company_final_masked.csv


,사번,이름,성별,나이,근속기간,입사년도,직급,팀구분,초과근무,성과점수,미사용.휴가일수,연봉,이메일,주민등록번호,전화번호
0,USER_01,Employee_0000,남,39,19,2015,사원,인사,213,42,14,66002987,han****@gmail.com,860921-*******,010-****-****
1,USER_02,Employee_0001,여,33,17,2012,사원,총무,324,49,9,58479133,img****@gmail.com,920608-*******,010-****-****
2,USER_03,Employee_0002,남,46,27,2013,사원,기획,192,40,22,46179481,igy****@gmail.com,791118-*******,010-****-****
3,USER_04,Employee_0003,남,39,19,2015,사원,영업,20,51,14,61002987,yun****@gmail.com,860819-*******,010-****-****
4,USER_05,Employee_0004,남,32,13,2011,주임,마켓팅,201,32,22,83001494,jug****@gmail.com,930626-*******,010-****-****
